In [1]:
#from sklearn.externals import joblib 
import joblib
from rdkit.Chem.Draw import IPythonConsole #Needed to show molecules+
from rdkit.Chem import Draw
from rdkit.Chem import AllChem as Chem
import gzip
import os
import sys
Type = sys.getfilesystemencoding()
#import sascorer as sa
from sklearn.ensemble import RandomForestClassifier
from syba.syba import SybaClassifier, SmiMolSupplier
import statsmodels.api as sm
from datetime import datetime
import tensorflow as tf

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [3]:
model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
model.add(layers.Embedding(input_dim=1024, output_dim=1))

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(280))

# Add a Dense layer with 10 units.
model.add(layers.Dense(100))

model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

model.summary()
model.save('./')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 1)           1024      
_________________________________________________________________
lstm (LSTM)                  (None, 280)               315840    
_________________________________________________________________
dense (Dense)                (None, 100)               28100     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 345,065
Trainable params: 345,065
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflo

In [32]:
test = [i for i in range(15)]
print(test[:5])
print(test[6:])

[0, 1, 2, 3, 4]
[6, 7, 8, 9, 10, 11, 12, 13, 14]


In [50]:
test_fps = []
test_mol = []
nBits = 1024
i = 0
for spls in SmiMolSupplier(gzip.open("../data/structures_2.csv.gz", mode="rt"), header=True, smi_col=1):
    test_fps.append(np.array(Chem.GetMorganFingerprintAsBitVect(spls[0],2,nBits=nBits)))
    test_mol.append(spls[2])
    i += 1
    if(i > 40):
        break;
print(test_mol[0])

Cc1ccsc1C(O)C1(C#N)Cc2ccccc2C1


In [55]:
non_classes = [0 for x in range(len(test_fps))]
fps = np.array(test_fps)
classes = np.array(non_classes)
n = len(fps)

x_train = np.concatenate((fps[:int(n*0.8)], fps))
y_train = classes[:int(n*0.8)]
x_test = fps[int(n*0.8):]
y_test = classes[int(n*0.8):]

result = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=10, epochs=1)
joblib.dump(result, "./lstm.pkl")


ValueError: Data cardinality is ambiguous:
  x sizes: 73
  y sizes: 32
Please provide data which shares the same first dimension.

In [ ]:
model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
model.add(layers.Embedding(input_dim=1000, output_dim=1))

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(128))

# Add a Dense layer with 10 units.
model.add(layers.Dense(10))

model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

#model.summary()


fichier = open("./logs/execution" + str(datetime.now()) + ".log", "a")
fichier.write("Début du calcule\n")

fichier.write(str(datetime.now()) + " 1\n")#1
nBits = 1024
fichier.write(str(datetime.now()) + " 2\n")#2
syn_fps = [Chem.GetMorganFingerprintAsBitVect(spls[0],2,nBits=nBits) for spls in SmiMolSupplier(gzip.open("./structures_2.csv.gz", mode="rt"), header=True, smi_col=1)]
syn_classes = [1 for x in range(len(syn_fps))]
fichier.write(str(datetime.now()) + " 3\n")#3
non_fps = [Chem.GetMorganFingerprintAsBitVect(spls[0],2,nBits=nBits) for spls in SmiMolSupplier(gzip.open("./structures_1.csv.gz", mode="rt"), header=True, smi_col=2)]
non_classes = [0 for x in range(len(non_fps))]
fichier.write(str(datetime.now()) + " 4\,")#4
fps = syn_fps + non_fps
classes = syn_classes + non_classes
n = len(fps)

x_train = fps[:int(n*0.8)]
y_train = classes[:int(n*0.8)]
x_test = fps[int(n*0.8):]
y_test = clases[int(n*0.8):]

result = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=batch_size, epochs=100)

fichier.write(str(datetime.now()) + " 5\n")#5
joblib.dump(result, "./rl.pkl")
fichier.write(str(datetime.now()) + " 7\n")#7

fichier.close()